In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
import keras
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

import numpy as np
import matplotlib.pyplot as plt

import pyarrow.parquet as pq
import pyarrow as pa

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, IncrementalPCA, SparsePCA
import pandas as pd
np.set_printoptions(suppress=True)

# Eugen stuff
import sklearn.svm
import sklearn

Using TensorFlow backend.


In [2]:
samples = pq.read_table('/data/gtex/samples.parquet').to_pandas()
genes = pd.read_csv("/data/gtex/genage_gtex.tsv", sep="\t", index_col=0, header=0)

In [3]:
data = genes.T
data.columns.names = ["Name"]
data.head(5)

Name,ENSG00000198804,ENSG00000196689,ENSG00000170899,ENSG00000259384,ENSG00000096968,ENSG00000112964,ENSG00000100311,ENSG00000127445,ENSG00000141837,ENSG00000118432,...,ENSG00000073756,ENSG00000143252,ENSG00000115966,ENSG00000169047,ENSG00000183395,ENSG00000017427,ENSG00000177606,ENSG00000164104,ENSG00000160957,ENSG00000059728
GTEX-1117F-0226-SM-5GZZ7,10660.0,20.67,24.83,0.2450,23.78,37.70,23.76,37.33,0.6694,6.685,...,2.1360,43.67,23.13,11.47,0.1898,40.63,526.4,139.50,3.667,16.590
GTEX-111CU-1826-SM-5GZYN,11060.0,10.58,17.71,0.4593,21.51,73.55,34.54,42.96,1.0900,29.800,...,0.5251,71.07,22.71,10.37,0.4575,68.23,256.6,95.25,3.141,12.520
GTEX-111FC-0226-SM-5N9B8,21320.0,15.66,30.25,4.6720,22.64,23.37,70.91,38.26,1.9000,10.750,...,1.1170,56.96,32.70,13.02,0.0000,24.38,519.3,76.09,3.941,8.683
GTEX-111VG-2326-SM-5N9BK,19850.0,12.31,16.50,4.8020,19.02,15.95,41.83,33.27,0.7103,1.704,...,6.1840,52.94,21.67,5.35,0.1227,29.26,871.3,69.30,5.417,37.350
GTEX-111YS-2426-SM-5GZZQ,31260.0,6.76,16.79,5.4420,31.18,116.20,71.35,27.91,1.0790,26.620,...,0.3932,44.61,23.08,10.55,0.8340,33.65,175.3,98.29,2.780,9.886


In [4]:
samples = samples.set_index("Name")
j = samples.join(data)
j.head(5)

,Subject,Tissue,Subtissue,Sex,Age,Death,Avg_age,ENSG00000198804,ENSG00000196689,ENSG00000170899,...,ENSG00000073756,ENSG00000143252,ENSG00000115966,ENSG00000169047,ENSG00000183395,ENSG00000017427,ENSG00000177606,ENSG00000164104,ENSG00000160957,ENSG00000059728
Name,,,,,,,,,,,,,,,,,,,,,
GTEX-11O72-2926-SM-5BC4V,GTEX-11O72,Brain,Brain - Cortex,1,40-49,2.0,44.5,80890.0,5.281,34.110,...,4.3410,20.27,19.450,2.518,0.0000,0.47010,93.79,21.86,6.599,8.137
GTEX-11TTK-0008-SM-5S2RU,GTEX-11TTK,Skin,Cells - Transformed fibroblasts,2,60-69,4.0,64.5,23710.0,4.964,43.160,...,0.8661,121.50,26.410,40.600,0.1677,0.08062,81.60,143.90,6.836,2.439
GTEX-11UD2-0226-SM-5EQKY,GTEX-11UD2,Skin,Skin - Sun Exposed (Lower leg),1,50-59,2.0,54.5,31680.0,5.967,65.190,...,4.8050,54.38,9.462,8.703,0.1313,1.18900,302.20,53.00,5.655,22.060
GTEX-12KS4-0005-SM-5SI94,GTEX-12KS4,Blood,Whole Blood,1,30-39,0.0,34.5,3867.0,1.801,1.044,...,10.1800,35.87,9.118,0.215,0.0000,0.32690,100.40,165.60,6.525,737.900
GTEX-131XE-2526-SM-5EQ57,GTEX-131XE,Nerve,Nerve - Tibial,1,50-59,0.0,54.5,12650.0,12.470,32.010,...,1.3920,69.37,25.880,10.210,0.3869,11.71000,535.00,97.57,12.400,16.790


In [5]:
ji = j.columns.drop(["Subject", "Tissue", "Subtissue", "Sex", "Age", "Death", "Avg_age"])

In [6]:
Y = j["Avg_age"].to_numpy()

In [7]:
X = j[ji].to_numpy()

In [8]:
x = keras.utils.normalize(X,axis=0)



In [9]:
(trainX, testX, trainY, testY) = train_test_split(x, Y.reshape(Y.shape[0],1), test_size=0.2, random_state=42)

In [10]:
trainX.shape

(9350, 297)

In [39]:
model = Sequential()
model.add(Dense(64, input_dim=trainX.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(48, input_dim=trainX.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(24, input_dim=trainX.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(12, activation="relu"))
model.add(Dense(1, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 64)                19072     
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 48)                3120      
_________________________________________________________________
dropout_13 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 24)                1176      
_________________________________________________________________
dropout_14 (Dropout)         (None, 24)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 12)                300       
__________

In [40]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.07
EPOCHS = 400

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])

[INFO] training network...


In [ ]:
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=256)

Train on 9350 samples, validate on 2338 samples
Epoch 1/400
9350/9350 [==============================] - 1s 75us/step - loss: 2918.9550 - mean_squared_error: 2918.9550 - val_loss: 2718.8330 - val_mean_squared_error: 2718.8330
Epoch 2/400
9350/9350 [==============================] - 0s 10us/step - loss: 2235.7342 - mean_squared_error: 2235.7342 - val_loss: 1103.2457 - val_mean_squared_error: 1103.2457
Epoch 3/400
9350/9350 [==============================] - 0s 10us/step - loss: 487.5827 - mean_squared_error: 487.5827 - val_loss: 297.5249 - val_mean_squared_error: 297.5249
Epoch 4/400
9350/9350 [==============================] - 0s 11us/step - loss: 315.0939 - mean_squared_error: 315.0939 - val_loss: 268.4393 - val_mean_squared_error: 268.4393
Epoch 5/400
9350/9350 [==============================] - 0s 11us/step - loss: 282.3030 - mean_squared_error: 282.3030 - val_loss: 236.8896 - val_mean_squared_error: 236.8896
Epoch 6/400
9350/9350 [==============================] - 0s 11us/step - lo

best results: 
-------------
0.1, 24-10% dropout-12-8-1 batch 64 and learning rate 0.1

126

In [ ]:
print("!!")